In [1]:
import tensorflow as tf
import numpy as np
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
import matplotlib.pyplot as plt


In [2]:
dataset_loc = 'drive/MyDrive/Dataset'

In [3]:
import shutil

dataset_local = "dataset"
num_classes=len(os.listdir(dataset_loc))
train = 0.95
val = 0.05

for folder in os.listdir(dataset_loc):
  os.makedirs(dataset_local+"/train/"+folder)
  os.makedirs(dataset_local+"/val/"+folder)

for folder in os.listdir(dataset_loc):
  num_files_in_folder=len(os.listdir(dataset_loc+'/'+folder))
  count=1
  for files in os.listdir(dataset_loc+"/"+folder):
    if count <= int(num_files_in_folder*train):
      shutil.copy(dataset_loc+"/"+folder+"/"+files, dataset_local+"/train/"+folder+"/")
      count=count+1
    else:
      shutil.copy(dataset_loc+"/"+folder+"/"+files, dataset_local+"/val/"+folder+"/")
      count=count+1

In [4]:
import tensorflow.keras.backend as K

K.clear_session()
n_classes = 29
img_width, img_height = 150, 150
train_dir = "dataset/train"
val_dir = "dataset/val"
batch_size = 16

In [5]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
  
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(directory=train_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode="categorical")
  
val_gen = test_datagen.flow_from_directory(directory=val_dir,
          target_size=(img_width, img_height),
          batch_size=batch_size,
          class_mode='categorical')

Found 12944 images belonging to 31 classes.
Found 696 images belonging to 31 classes.


In [20]:
from tensorflow.keras import regularizers

inception = InceptionV3(input_shape=(150,150,3),weights='imagenet', include_top=False)
for i in inception.layers:
  i.trainable=False
x = inception.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

predictions = tf.keras.layers.Dense(31,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

In [22]:
model = tf.keras.models.Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_376 (Conv2D)            (None, 74, 74, 32)   864         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_376 (Batch  (None, 74, 74, 32)  96          ['conv2d_376[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [ ]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=25 )

Epoch 1/25
809/809 [==============================] - 711s 878ms/step - loss: 1.8337 - accuracy: 0.6051 - val_loss: 1.5048 - val_accuracy: 0.6882
Epoch 2/25
809/809 [==============================] - 700s 865ms/step - loss: 1.5219 - accuracy: 0.6712 - val_loss: 1.3374 - val_accuracy: 0.7198
Epoch 3/25
809/809 [==============================] - 712s 879ms/step - loss: 1.4011 - accuracy: 0.7009 - val_loss: 1.2429 - val_accuracy: 0.7342
Epoch 4/25
809/809 [==============================] - 711s 879ms/step - loss: 1.3174 - accuracy: 0.7138 - val_loss: 1.2042 - val_accuracy: 0.7356
Epoch 5/25
809/809 [==============================] - 718s 888ms/step - loss: 1.2557 - accuracy: 0.7287 - val_loss: 1.1498 - val_accuracy: 0.7457
Epoch 6/25
809/809 [==============================] - 711s 879ms/step - loss: 1.1998 - accuracy: 0.7361 - val_loss: 1.0835 - val_accuracy: 0.7658
Epoch 7/25
809/809 [==============================] - 711s 878ms/step - loss: 1.1582 - accuracy: 0.7446 - val_loss: 1.0440 -

In [ ]:
model.save('model_trained_29class.h5')

In [ ]:
def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()

def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()

plot_accuracy(history,'IndonesianFood-Inceptionv3')
plot_loss(history,'IndonesianFood-Inceptionv3')

In [ ]:
class_names = train_gen.class_indices

class_names

In [ ]:
from tensorflow.keras.models import load_model
# Loading the best saved model to make predictions
K.clear_session()
model_best = load_model('model_trained_29class.h5',compile = False)

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_class(model, images, show = True):
  for img in images:
    img = image.load_img(img, target_size=(150, 150))
    img = image.img_to_array(img)                    
    img = np.expand_dims(img, axis=0)/255                                   
    pred = model.predict(img)
    index = np.argmax(pred)
    pred=list(class_names.keys())[list(class_names.values()).index(index)]
    if show:
        plt.imshow(img[0])                           
        plt.axis('off')
        plt.title(pred)
        plt.show()

images = []
images.append('apacoba.jpg')
images.append('tes2.jpg')
images.append('bakso (13).jpg')
predict_class(model_best, images, True)